# **Malicious URL Detect**

#### malicious-url-detection-with-ML

malicious url detection with machine learning

In [9]:
import os
import json
from pathlib import Path
import pandas as pd
import numpy as np

from string import printable
from sklearn import model_selection

from keras.preprocessing import sequence
from keras.models import model_from_json

In [10]:
def print_layers_dims(model):
    l_layers = model.layers
    # Note None is ALWAYS batch_size
    for i in range(len(l_layers)):
        print(l_layers[i])
        print('Input Shape: ', l_layers[i].input_shape,
              'Output Shape: ', l_layers[i].output_shape)

In [11]:
def save_model(model, fileModelJSON, fileWeights):
    # have h5py installed
    if Path(fileModelJSON).is_file():
        os.remove(fileModelJSON)
    json_string = model.to_json()
    with open(fileModelJSON, 'w') as f:
        json.dump(json_string, f)

    if Path(fileWeights).is_file():
        os.remove(fileWeights)
    model.save_weights(fileWeights)


def load_model(fileModelJSON, fileWeights):
    with open(fileModelJSON, 'r') as f:
        model_json = json.load(f)
        model = model_from_json(model_json)

    model.load_weights(fileWeights)
    return model

## read data

In [12]:
def read_data():
    data_file_path = os.path.join("data/malicious_url", 'url_data.csv')
    df = pd.read_csv(data_file_path)

    # Step 1: Convert raw URL string in list of lists where characters that are contained in "printable" are stored encoded as integer
    url_int_tokens = [
        [printable.index(x) + 1 for x in url if x in printable] for url in df.url]

    # Step 2: Cut URL string at max_len or pad with zeros if shorter
    max_len = 75
    X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)

    # Step 3: Extract labels form df to numpy array
    target = np.array(df.isMalicious)

    print('Matrix dimensions of X: ', X.shape,
          'Vector dimension of target: ', target.shape)

    X_train, X_test, target_train, target_test = model_selection.train_test_split(
        X, target, test_size=0.25, random_state=33)

    return X_train, X_test, target_train, target_test

In [13]:
X_train, X_test, target_train, target_test = read_data()
X_train

Matrix dimensions of X:  (194798, 75) Vector dimension of target:  (194798,)


array([[ 0,  0,  0, ..., 32, 24, 21],
       [76, 26, 18, ..., 24, 30, 25],
       [62, 68, 11, ..., 35, 81, 31],
       ...,
       [ 0,  0,  0, ..., 24, 15, 22],
       [ 0,  0,  0, ...,  4, 77,  3],
       [ 0,  0,  0, ..., 36, 19, 30]], dtype=int32)

In [14]:
from models.cnn import ConvFully
from models.rnn import SimpleLSTM
from models.cnn_rnn import ConvLSTM
epochs_num= 10
batch_size=32 #8

## **RNN**

In [16]:
model_name = "simple_lstm"
model = SimpleLSTM().build_model()
model.fit(X_train, target_train,
            epochs=epochs_num, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)

print('\nFinal Cross-Validation Accuracy', accuracy, '\n')

print_layers_dims(model)
# save model
json_file_path = os.path.join("cache/malicious_url", model_name + ".json")
weight_file_path = os.path.join("cache/malicious_url", model_name + ".h5")
save_model(model, json_file_path, weight_file_path)

Epoch 1/10
4566/4566 [==============================] - 80s 17ms/step - loss: 0.5408 - accuracy: 0.7331
Epoch 2/10
4566/4566 [==============================] - 77s 17ms/step - loss: 0.4144 - accuracy: 0.8200
Epoch 3/10
4566/4566 [==============================] - 76s 17ms/step - loss: 0.3971 - accuracy: 0.8267
Epoch 4/10
4566/4566 [==============================] - 75s 16ms/step - loss: 0.3871 - accuracy: 0.8319
Epoch 5/10
4566/4566 [==============================] - 75s 16ms/step - loss: 0.3791 - accuracy: 0.8374
Epoch 6/10
4566/4566 [==============================] - 77s 17ms/step - loss: 0.3673 - accuracy: 0.8434
Epoch 7/10
4566/4566 [==============================] - 79s 17ms/step - loss: 0.3595 - accuracy: 0.8460
Epoch 8/10
4566/4566 [==============================] - 77s 17ms/step - loss: 0.3534 - accuracy: 0.8505
Epoch 9/10
4566/4566 [==============================] - 78s 17ms/step - loss: 0.3458 - accuracy: 0.8546
Epoch 10/10
1522/1522 [==============================] - 6s 4ms/

## **CNN**

In [17]:
model_name = "conv_fully"

model = ConvFully().build_model()
model.fit(X_train, target_train,
            epochs=epochs_num, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)

print('\nFinal Cross-Validation Accuracy', accuracy, '\n')

print_layers_dims(model)
# save model
json_file_path = os.path.join("cache/malicious_url", model_name + ".json")
weight_file_path = os.path.join("cache/malicious_url", model_name + ".h5")
save_model(model, json_file_path, weight_file_path)


Epoch 1/10
4566/4566 [==============================] - 179s 39ms/step - loss: 0.7257 - accuracy: 0.6510
Epoch 2/10
4566/4566 [==============================] - 210s 46ms/step - loss: 0.5036 - accuracy: 0.7558
Epoch 3/10
2097/4566 [============>.................] - ETA: 1:58 - loss: 0.4673 - accuracy: 0.7791

KeyboardInterrupt: 

## **Conv_LSTM**

In [ ]:
model_name = "conv_lstm"

model = ConvLSTM().build_model()
model.fit(X_train, target_train,
            epochs=epochs_num, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)

print('\nFinal Cross-Validation Accuracy', accuracy, '\n')

print_layers_dims(model)
# save model
json_file_path = os.path.join("cache/malicious_url", model_name + ".json")
weight_file_path = os.path.join("cache/malicious_url", model_name + ".h5")
save_model(model, json_file_path, weight_file_path)